In [1]:
%matplotlib inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets,layers,models
import numpy as np
import matplotlib.pyplot as plt


In [10]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


In [11]:
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10
print('Noof classes:',K)
print('Noof training images:',Ntr)
print('Noof test images:',Nte)
print('Noof features:',Din)

x_train=x_train[range(Ntr),:]
y_train=y_train[range(Ntr),:]
print(y_train.shape)
x_test=x_test[range(Nte),:]
x_test=x_test[range(Nte),:]


Noof classes: 10
Noof training images: 50000
Noof test images: 10000
Noof features: 3072
(50000, 1)


In [12]:
print('original labels:',y_train.shape)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
print('labels in class matrix:',y_train.shape)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)

x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
print('x_train:', x_train.shape)
print('x_test:', x_test.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize pixel values
#x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)
x_train = x_train - mean_image
x_test = x_test - mean_image



original labels: (50000, 1)
labels in class matrix: (50000, 10)
x_train: (50000, 3072)
x_test: (10000, 3072)


Two Layer Fully Connected Network

In [13]:
H=200
std=1e-6
w1 = std*np.random.randn(Din, H)
w2 = std*np.random.randn(H, K)
b1 = np.zeros(H)
b2=np.zeros(K)
print("w1:", w1.shape)
print("b1:", b1.shape)
print("w2:", w2.shape)
print("b2:", b2.shape)

w1: (3072, 200)
b1: (200,)
w2: (200, 10)
b2: (10,)


Stochastic Gradient Descent (Going through batches in order)

In [15]:
batch_size = 500
step=Ntr/batch_size
iterations = 300
lr = 1.4e-2
lr_decay=0.999
reg =5e-6
train_loss_history = []
train_acc_history = []
test_loss_history = []
test_acc_history = []
seed = 0

for t in range(iterations):
    start=int(batch_size*((t)%step))
    x=x_train[start:start+batch_size,:]
    y=y_train[start:start+batch_size,:]

    # Forward pass
    z1=1.0/(1.0 + np.exp(-(x.dot(w1)+b1)))
    y_pred=z1.dot(w2)+b2
    z=1.0/(1.0 + np.exp(-(x_test.dot(w1)+b1)))
    y_pred_test=z.dot(w2)+b2

    #loss
    loss=(1.0/batch_size)*np.square(y_pred-y).sum() + reg * (np.sum(w1*w1) + np.sum(w2*w2))
    test_loss=(1.0/Nte)*np.square(y_pred_test-y_test).sum() + reg * (np.sum(w1*w1) + np.sum(w2*w2))
    train_loss_history.append(loss)
    test_loss_history.append(test_loss)

    #accuracy
    train_acc = 1.0 - (1/(batch_size*K))*(np.abs(np.argmax(y,axis=1) - np.argmax(y_pred,axis=1))).sum()
    train_acc_history.append(train_acc)
    test_acc = 1.0 - (1/(Nte*K))*(np.abs(np.argmax(y_test,axis=1) - np.argmax(y_pred_test,axis=1))).sum()
    test_acc_history.append(test_acc)

    if t%10==0:
        print('epoch %d/%d: train loss= %f-- ,test loss= %f--,train accuracy= %f--, test accuracy= %f' % (t,iterations,loss,test_loss,train_acc,test_acc))

        
    # Backward pass

    dy_pred=(1./batch_size)*2.0*(y_pred-y)
    dw2=z1.T.dot(dy_pred) + reg * w2
    db2=dy_pred.sum(axis=0)
    dz1=dy_pred.dot(w2.T)
    dw1=x.T.dot(dz1*z1*(1-z1))+reg*w1
    db1=(dz1*z1*(1-z1)).sum(axis=0)


    
    w1 -= lr*dw1
    w2 -= lr*dw2
    b1 -= lr*db1
    b2 -= lr*db2
    lr *= lr_decay






epoch 0/300: train loss= 0.747635-- ,test loss= 0.771751--,train accuracy= 0.795200--, test accuracy= 0.780480
epoch 10/300: train loss= 0.782559-- ,test loss= 0.779988--,train accuracy= 0.770400--, test accuracy= 0.774110
epoch 20/300: train loss= 0.773794-- ,test loss= 0.775199--,train accuracy= 0.770600--, test accuracy= 0.782600
epoch 30/300: train loss= 0.779001-- ,test loss= 0.775631--,train accuracy= 0.792600--, test accuracy= 0.784080
epoch 40/300: train loss= 0.783227-- ,test loss= 0.775139--,train accuracy= 0.779200--, test accuracy= 0.788540
epoch 50/300: train loss= 0.751493-- ,test loss= 0.767492--,train accuracy= 0.801200--, test accuracy= 0.782460
epoch 60/300: train loss= 0.763654-- ,test loss= 0.771161--,train accuracy= 0.762200--, test accuracy= 0.780100
epoch 70/300: train loss= 0.765465-- ,test loss= 0.772608--,train accuracy= 0.788000--, test accuracy= 0.781350
epoch 80/300: train loss= 0.764949-- ,test loss= 0.772618--,train accuracy= 0.775400--, test accuracy= 0.

Stochastic Gradient Descent (Going through batches in a random order)

In [21]:
batch_size = 500
step=Ntr/batch_size
iterations = 300
lr = 1.4e-2
lr_decay=0.999
reg =5e-6
train_loss_history = []
train_acc_history = []
test_loss_history = []
test_acc_history = []
seed = 0
rng = np.random.default_rng(seed=seed)
indices=np.arange(0,step)
for t in range(iterations):
    rng.shuffle(indices)
    for order in range(0,len(indices)):        
        start=int(batch_size*indices[order])
        x=x_train[start:start+batch_size,:]
        y=y_train[start:start+batch_size,:]
        

        # Forward pass
        z1=1.0/(1.0 + np.exp(-(x.dot(w1)+b1)))
        y_pred=z1.dot(w2)+b2
        z=1.0/(1.0 + np.exp(-(x_test.dot(w1)+b1)))
        y_pred_test=z.dot(w2)+b2

        #loss
        loss=(1.0/batch_size)*np.square(y_pred-y).sum() + reg * (np.sum(w1*w1) + np.sum(w2*w2))
        test_loss=(1.0/Nte)*np.square(y_pred_test-y_test).sum() + reg * (np.sum(w1*w1) + np.sum(w2*w2))
        train_loss_history.append(loss)
        test_loss_history.append(test_loss)

        #accuracy
        train_acc = 1.0 - (1/(batch_size*K))*(np.abs(np.argmax(y,axis=1) - np.argmax(y_pred,axis=1))).sum()
        train_acc_history.append(train_acc)
        test_acc = 1.0 - (1/(Nte*K))*(np.abs(np.argmax(y_test,axis=1) - np.argmax(y_pred_test,axis=1))).sum()
        test_acc_history.append(test_acc)

        if t%10==0 and order%20==0:
            print('epoch %d/%d: ' % (t,iterations))
            print('train loss= %f-- ,test loss= %f--,train accuracy= %f--, test accuracy= %f' % (loss,test_loss,train_acc,test_acc))

                    
        # Backward pass

        dy_pred=(1./batch_size)*2.0*(y_pred-y)
        dw2=z1.T.dot(dy_pred) + reg * w2
        db2=dy_pred.sum(axis=0)
        dz1=dy_pred.dot(w2.T)
        dw1=x.T.dot(dz1*z1*(1-z1))+reg*w1
        db1=(dz1*z1*(1-z1)).sum(axis=0)
    
        w1 -= lr*dw1
        w2 -= lr*dw2
        b1 -= lr*db1
        b2 -= lr*db2
        lr *= lr_decay






epoch 0/300: 
train loss= 0.716159-- ,test loss= 0.746209--,train accuracy= 0.809000--, test accuracy= 0.793060
epoch 0/300: 
train loss= 0.726833-- ,test loss= 0.766283--,train accuracy= 0.797600--, test accuracy= 0.783790
epoch 0/300: 
train loss= 0.754795-- ,test loss= 0.777773--,train accuracy= 0.781800--, test accuracy= 0.768990
epoch 0/300: 
train loss= 0.722950-- ,test loss= 0.755010--,train accuracy= 0.795600--, test accuracy= 0.786830
epoch 0/300: 
train loss= 0.720323-- ,test loss= 0.759143--,train accuracy= 0.813600--, test accuracy= 0.789960
epoch 10/300: 
train loss= 0.644920-- ,test loss= 0.748028--,train accuracy= 0.847000--, test accuracy= 0.797090
epoch 10/300: 
train loss= 0.653220-- ,test loss= 0.747254--,train accuracy= 0.861800--, test accuracy= 0.795560
epoch 10/300: 
train loss= 0.634627-- ,test loss= 0.748866--,train accuracy= 0.851000--, test accuracy= 0.797200
epoch 10/300: 
train loss= 0.645471-- ,test loss= 0.747715--,train accuracy= 0.857800--, test accurac